In [8]:
# Module Importations
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
# Project Module Importations
from GensetSimulation.data_analytics.src.data import cosmos_client_manager
from GensetSimulation.data_analytics.src.data import load_data
from GensetSimulation.data_analytics.src.features import data_munging

In [3]:
# Constants (i.e. for saving figures)

In [3]:
# Load data from database
client = cosmos_client_manager.intialise_client()
database = cosmos_client_manager.return_database(client)
container = cosmos_client_manager.return_container(database)
item_list = cosmos_client_manager.read_items(container)

Database with id 'BMRS Data' was found, link is dbs/BMRS Data
Container with id 'DataElement' was found, link is dbs/BMRS Data/colls/DataElement
Reading all items in container
Found 22924 items


In [4]:
# Convert to dataframe
df_items = load_data.convert_dict_to_dataframe(item_list)

In [11]:
# Save dataframe locally
load_data.save_local_dataframe(df_items)

Saved dataframe locally ...


In [3]:
df_new = load_data.load_local_dataframe()

Loaded local dataframe ...


In [6]:
df_final = data_munging.remove_unused_columns(df_new)

In [7]:
print(df_final.head())

id                     docType           busType  \
0          2020-08-01-P01-Solar  Actual generation per type  Solar generation   
1  2020-08-01-P01-Wind Offshore  Actual generation per type   Wind generation   
2   2020-08-01-P01-Wind Onshore  Actual generation per type   Wind generation   
3          2020-08-01-P01-Other  Actual generation per type        Production   
4        2020-08-01-P01-Nuclear  Actual generation per type        Production   

    proType                       timeId  quantity  \
0  Realised  NGET-EMFIP-AGPT-TS-21630860         1   
1  Realised  NGET-EMFIP-AGPT-TS-21630861  4509.018   
2  Realised  NGET-EMFIP-AGPT-TS-21630862  1721.168   
3  Realised  NGET-EMFIP-AGPT-TS-21630863       144   
4  Realised  NGET-EMFIP-AGPT-TS-21630864      5293   

                     curveType resolution     setDate setPeriod  \
0  Sequential fixed size block      PT30M  2020-08-01         1   
1  Sequential fixed size block      PT30M  2020-08-01         1   
2  Sequential fi

In [ ]:
# Plot generation data

plt.figure(figsize =(14, 10))


# Plot solar and wind generation
for generation_type, data in df_final.groupby('powType'):
    plt.scatter(data['id'], data[])